In [ ]:
from ztmwarsaw.api.bus_caller import BusCaller

In [ ]:
buscaller = BusCaller()
buscaller.build(apikey="54b839d9-364d-4877-ae15-a624352175a3")
buscaller.call()

https://api.um.warszawa.pl/api/action/datastore_search?resource_id=54b839d9-364d-4877-ae15-a624352175a3&apikey=54b839d9-364d-4877-ae15-a624352175a3


{'result': 'Błędna metoda lub parametry wywołania'}

In [ ]:
import requests

url = "https://api.um.warszawa.pl/api/action/busestrams_get/"
params = {
    "resource_id": "f2e5503e-927d-4ad3-9500-4ab9e55deb59",
    "apikey": "54b839d9-364d-4877-ae15-a624352175a3",
    "type": 1,
    "line": 157
}
r = requests.get(url=url, params=params)
response = r.json()
print(response)

{'result': [{'Lines': '157', 'Lon': 20.976549, 'VehicleNumber': '9424', 'Time': '2024-01-11 22:16:24', 'Lat': 52.289917, 'Brigade': '3'}, {'Lines': '157', 'Lon': 20.982937, 'VehicleNumber': '9426', 'Time': '2024-01-11 22:16:20', 'Lat': 52.248955, 'Brigade': '5'}, {'Lines': '157', 'Lon': 20.980892, 'VehicleNumber': '9429', 'Time': '2024-01-11 22:16:21', 'Lat': 52.275276, 'Brigade': '430'}, {'Lines': '157', 'Lon': 21.015215, 'VehicleNumber': '9432', 'Time': '2024-01-11 22:16:21', 'Lat': 52.318569, 'Brigade': '4'}, {'Lines': '157', 'Lon': 21.01499, 'VehicleNumber': '9440', 'Time': '2024-01-11 22:11:40', 'Lat': 52.318089, 'Brigade': '9'}, {'Lines': '157', 'Lon': 20.981087, 'VehicleNumber': '9441', 'Time': '2024-01-11 22:16:24', 'Lat': 52.216572, 'Brigade': '8'}, {'Lines': '157', 'Lon': 20.98484, 'VehicleNumber': '9442', 'Time': '2024-01-11 22:16:22', 'Lat': 52.246758, 'Brigade': '10'}, {'Lines': '157', 'Lon': 21.015392, 'VehicleNumber': '9443', 'Time': '2024-01-11 20:29:15', 'Lat': 52.3190